# Задание

### Реализовать индексы, повышающие производительность операций вставки и изменения платежей без модификации программных компонент

Подключаемся к БД

In [1]:
import pyodbc 

str_param = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=PaymentData;'
    r'Trusted_Connection=yes;'
    r'Encrypt=no;'
    )
cnxn = pyodbc.connect(str_param)
cursor = cnxn.cursor()

Для начала проверим время испольнения запросов `INSERT` и `UPDATE` до введения индексов

In [2]:
from random import randint


def get_participant_oid():
    query = """
    SELECT TOP 1 Oid
    FROM PaymentParticipant
    ORDER BY NEWID() 
    """
    return cursor.execute(query).fetchval()


def get_category_oid():
    query = """
    SELECT TOP 1 Oid
    FROM PaymentCategory
    ORDER BY NEWID() 
    """
    return cursor.execute(query).fetchval()


def get_project_oid():
    query = """
    SELECT TOP 1 Oid
    FROM Project
    ORDER BY NEWID() 
    """
    return cursor.execute(query).fetchval()


def get_payment_oid():
    query = """
    SELECT TOP 1 Oid
    FROM Payment
    ORDER BY NEWID() 
    """
    return cursor.execute(query).fetchval()

In [3]:
%%timeit -r 1 -n 1
for _ in range(2000):
    payer_oid = get_participant_oid()
    payee_oid = get_participant_oid()
    if payee_oid == payee_oid:
        payee_oid = get_participant_oid()
    payment_category_oid = get_category_oid()
    project_oid = get_project_oid()

    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
    Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
    CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), '{randint(0, 42000)}', '{payment_category_oid}', '{project_oid}', NULL,
    'Random inserted payment', NULL, '{payer_oid}', '{payee_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
cursor.commit()

1.46 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [4]:
%%timeit -r 1 -n 1
for _ in range(2000):
    payer_oid = get_participant_oid()
    payee_oid = get_participant_oid() 
    if payee_oid == payee_oid:
        payee_oid = get_participant_oid()
    payment_category_oid = get_category_oid()
    payment_oid = get_payment_oid()

    query = f"""
    UPDATE Payment
    SET Amount = Amount + 42, Payer = '{payer_oid}', Payee = '{payee_oid}', Category = '{payment_category_oid}'
    WHERE Oid = '{payment_oid}'
    """
    cursor.execute(query)
cursor.commit()

1.86 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Теперь добавим индексы к нашей БД

In [5]:
indexes = [
    "NONCLUSTERED INDEX iName_PaymentCategory ON PaymentCategory (Name)",
    "NONCLUSTERED INDEX iName_AccountType ON AccountType (Name)",
    "NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON Supplier (ProfitByMaterialAsPayer)",
    "NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON Supplier (ProfitByMaterialAsPayee)",
    "NONCLUSTERED INDEX iProfitByMaterial_PaymentCategory ON PaymentCategory (ProfitByMaterial)",
    "NONCLUSTERED INDEX iCostByMaterial_PaymentCategory ON PaymentCategory (CostByMaterial)",
    "NONCLUSTERED INDEX iNotInPaymentParticipantProfit_PaymentCategory ON PaymentCategory (NotInPaymentParticipantProfit)",
]

for index in indexes:
    query = f"""
    CREATE {index}
    """
    cursor.execute(query)
    cursor.commit()

In [6]:
%%timeit -r 1 -n 1
for _ in range(2000):
    payer_oid = get_participant_oid()
    payee_oid = get_participant_oid()
    if payee_oid == payee_oid:
        payee_oid = get_participant_oid()
    payment_category_oid = get_category_oid()
    project_oid = get_project_oid()

    query = f"""
    INSERT INTO Payment
    (Oid, Amount, Category, Project, Justification,
    Comment, Date, Payer, Payee, OptimisticLockField, GCRecord,
    CreateDate, CheckNumber, IsAuthorized, Number)
    VALUES
    (NEWID(), '{randint(0, 42000)}', '{payment_category_oid}', '{project_oid}', NULL,
    'Random inserted payment', NULL, '{payer_oid}', '{payee_oid}', NULL, NULL,
    NULL, NULL, NULL, NULL)
    """
    cursor.execute(query)
cursor.commit()

1.04 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%%timeit -r 1 -n 1
for _ in range(2000):
    payer_oid = get_participant_oid()
    payee_oid = get_participant_oid() 
    if payee_oid == payee_oid:
        payee_oid = get_participant_oid()
    payment_category_oid = get_category_oid()
    payment_oid = get_payment_oid()

    query = f"""
    UPDATE Payment
    SET Amount = Amount + 42, Payer = '{payer_oid}', Payee = '{payee_oid}', Category = '{payment_category_oid}'
    WHERE Oid = '{payment_oid}'
    """
    cursor.execute(query)
cursor.commit()

1.60 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
